# Running Tune experiments with BayesOpt

This example demonstrates the usage of BayesOpt with Ray Tune, including conditional search spaces.

We also combine the search algorithm based on `BayesOptSearch` with `AsyncHyperBandScheduler` scheduler to demonstrate Ray Tune's modularity.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

Necessary requirements:
- `pip install ray[tune]`
- `pip install bayesian-optimization==1.2.0`

In [ ]:
# !pip install ray[tune]
!pip install bayesian-optimization==1.2.0

In [ ]:
import time

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`.

In [ ]:
def evaluate(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100) ** (-1) + height * 0.1

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Now we define the search algorithm built from `BayesOptSearch`, constrained  to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`.

In [ ]:
algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
algo = ConcurrencyLimiter(algo, max_concurrent=4)

Furthermore, we define a `scheduler` to go along with our algorithm to showcase the modularity of Ray Tune.

In [ ]:
scheduler = AsyncHyperBandScheduler()

The number of samples this Tune run is set to `1000`.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is to define a search space.

In [ ]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
}

And run the experiment.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="bayesopt_exp",
    num_samples=num_samples,
    config=search_space,
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
ray.shutdown()